In [1]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [2]:
from utils.functions import *

In [3]:
data = TimeSeriesDataFrame.from_path(
    "https://autogluon.s3.amazonaws.com/datasets/timeseries/grocery_sales/test.csv",
)
data.head()

scaled_price  promotion_email  promotion_homepage  \
item_id  timestamp                                                       
1062_101 2018-01-01      0.879130              0.0                 0.0   
         2018-01-08      0.994517              0.0                 0.0   
         2018-01-15      1.005513              0.0                 0.0   
         2018-01-22      1.000000              0.0                 0.0   
         2018-01-29      0.883309              0.0                 0.0   

                     unit_sales  
item_id  timestamp               
1062_101 2018-01-01       636.0  
         2018-01-08       123.0  
         2018-01-15       391.0  
         2018-01-22       339.0  
         2018-01-29       661.0

In [4]:
prediction_length = 8
train_data, test_data = data.train_test_split(prediction_length=prediction_length)

In [5]:
predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    target="unit_sales",
    known_covariates_names=["scaled_price", "promotion_email", "promotion_homepage"],
).fit(
    train_data,
    hyperparameters={
        "Chronos": [
            # Zero-shot model WITHOUT covariates
            {
                "model_path": "bolt_small",
                "ag_args": {"name_suffix": "ZeroShot"},
            },
            # Chronos-Bolt (Small) combined with CatBoost on covariates
            {
                "model_path": "bolt_small",
                "covariate_regressor": "CAT",
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "WithRegressor"},
            },
        ],
    },
    enable_ensemble=False,
    time_limit=60,
)

Beginning AutoGluon training... Time limit = 60s
AutoGluon will save models to '/home/antares/Documents/PoliTo/Fall_24/DeepNLP/DeepNLP_Forecasting/AutogluonModels/ag-20250517_152546'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.9
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #202412060638~1743109366~22.04~1fce33b SMP PREEMPT_DYNAMIC Thu M
CPU Count:          16
GPU Count:          1
Memory Avail:       7.25 GB / 15.32 GB (47.3%)
Disk Space Avail:   319.98 GB / 441.91 GB (72.4%)

Fitting with arguments:
{'enable_ensemble': False,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': [{'ag_args': {'name_suffix': 'ZeroShot'},
                                  'model_path': 'bolt_small'},
                                 {'ag_args': {'name_suffix': 'WithRegressor'},
                                  'covariate_regressor': 'CAT',
                                  'model_path': 'bolt_small',
                   

In [6]:
predictor.leaderboard(test_data)

Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time_marginal,fit_order
0,ChronosWithRegressor[bolt_small],-0.268969,-0.358048,1.024099,0.822945,0.502212,2
1,ChronosZeroShot[bolt_small],-0.318562,-0.452296,1.079372,1.361618,2.289324,1


In [ ]:
covars = test_data.drop('unit_sales', axis=1, inplace=False)
covars.index.get_level_values('item_id')

Index(['1062_101', '1062_101', '1062_101', '1062_101', '1062_101', '1062_101',
       '1062_101', '1062_101', '1062_101', '1062_101',
       ...
       '2956_108', '2956_108', '2956_108', '2956_108', '2956_108', '2956_108',
       '2956_108', '2956_108', '2956_108', '2956_108'],
      dtype='object', name='item_id', length=9889)

In [34]:
covars.loc['1062_101']

,scaled_price,promotion_email,promotion_homepage
timestamp,,,
2018-01-01,0.879130,0.0,0.0
2018-01-08,0.994517,0.0,0.0
2018-01-15,1.005513,0.0,0.0
2018-01-22,1.000000,0.0,0.0
2018-01-29,0.883309,0.0,0.0
2018-02-05,0.872894,0.0,0.0
2018-02-12,0.878020,0.0,0.0
2018-02-19,0.888481,0.0,0.0
2018-02-26,0.988978,0.0,0.0


In [33]:
predictions3 = predictor.predict(
    test_data.loc['1062_101'].iloc[: -prediction_length], known_covariates=covars.loc['1062_101'].iloc[-prediction_length:])

KeyError: 'Requested level (item_id) does not match index name (timestamp)'

In [ ]:
test_data.loc[:,]

scaled_price  promotion_email  promotion_homepage  \
item_id  timestamp                                                       
1062_101 2018-01-01      0.879130              0.0                 0.0   
         2018-01-08      0.994517              0.0                 0.0   
         2018-01-15      1.005513              0.0                 0.0   
         2018-01-22      1.000000              0.0                 0.0   
         2018-01-29      0.883309              0.0                 0.0   
...                           ...              ...                 ...   
2956_108 2018-07-02      1.001721              0.0                 0.0   
         2018-07-09      0.998282              0.0                 0.0   
         2018-07-16      0.996570              0.0                 0.0   
         2018-07-23      1.000000              0.0                 0.0   
         2018-07-30      1.001718              0.0                 0.0   

                     unit_sales  
item_id  timestamp               
1062_101 2018-01-01       636.0  
         2018-01-08       123.0  
         2018-01-15       391.0  
         2018-01-22       339.0  
         2018-01-29       661.0  
...                         ...  
2956_108 2018-07-02        15.0  
         2018-07-09        67.0  
         2018-07-16        82.0  
         2018-07-23        96.0  
         2018-07-30        68.0  

[9889 rows x 4 columns]